# FORMATIVE ASSIGNMENT: ADVANCED LINEAR ALGEBRA (PCA)

## Principal Component Analysis Implementation

**Student Name:** Haguma Bianca

**Date:** February 2026



## Assignment Overview
This notebook implements Principal Component Analysis (PCA) from scratch using eigenvalue decomposition and covariance matrices. The goal is to reduce dimensionality while preserving maximum variance in the data.

### Learning Objectives:
1. Understand and implement covariance matrix calculation
2. Perform eigendecomposition to find principal components
3. Project data onto principal components
4. Select components based on explained variance
5. Visualize the transformation

## Part 0: Import Required Libraries

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA as SklearnPCA
import warnings
import time
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## Part 1: Data Loading and Exploration

### Data Requirements:
- Must have missing values (NaN)
- Must have at least 1 non-numeric column
- Must have more than 10 columns
- Should be impactful African/Africanized data

### Example Dataset Suggestions:
- African Economic Development Indicators
- Healthcare/Disease Data from African Countries
- Agricultural Production Data
- Climate and Weather Data
- Education Statistics

In [ ]:
# TO DO: Load your dataset
# Replace 'your_data.csv' with your actual data file
# Example: df = pd.read_csv('african_health_data.csv')

df = pd.read_csv('your_data.csv')

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nColumns ({len(df.columns)}):")
print(df.columns.tolist())

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Basic information about the dataset
print("Dataset Information:")
print("="*50)
df.info()
print("\n" + "="*50)
print("\nBasic Statistics:")
df.describe()

## Part 2: Data Preprocessing

### This section covers:
1. Identifying missing values
2. Identifying non-numeric columns
3. Handling missing values
4. Encoding categorical variables

In [ ]:
# Check for missing values
print("Missing Values Analysis:")
print("="*50)
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Percentage': missing_percent.values
})

missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
print(missing_df.to_string(index=False))

# Visualize missing values
if len(missing_df) > 0:
    plt.figure(figsize=(10, 6))
    plt.barh(missing_df['Column'], missing_df['Percentage'])
    plt.xlabel('Percentage of Missing Values')
    plt.title('Missing Values by Column')
    plt.tight_layout()
    plt.show()

In [ ]:
# Identify non-numeric columns
print("Data Types Analysis:")
print("="*50)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print(f"\nNumeric columns ({len(numeric_cols)}):")
print(numeric_cols)
print(f"\nNon-numeric columns ({len(non_numeric_cols)}):")
print(non_numeric_cols)

In [ ]:
# TO DO: Handle missing values
# You can use various strategies:
# 1. Mean/Median imputation for numeric columns
# 2. Mode imputation for categorical columns
# 3. Forward/Backward fill
# 4. Drop rows/columns with too many missing values

# Create a copy of the dataframe
df_processed = df.copy()

# Strategy 1: For numeric columns, fill with median
for col in numeric_cols:
    if df_processed[col].isnull().any():
        median_value = df_processed[col].median()
        df_processed[col].fillna(median_value, inplace=True)
        print(f"Filled {col} with median: {median_value:.2f}")

# Strategy 2: For categorical columns, fill with mode
for col in non_numeric_cols:
    if df_processed[col].isnull().any():
        mode_value = df_processed[col].mode()[0] if len(df_processed[col].mode()) > 0 else 'Unknown'
        df_processed[col].fillna(mode_value, inplace=True)
        print(f"Filled {col} with mode: {mode_value}")

print("\nMissing values after imputation:")
print(df_processed.isnull().sum().sum())

In [ ]:
# TO DO: Encode non-numeric columns
# Use Label Encoding or One-Hot Encoding depending on the nature of categorical variables

df_encoded = df_processed.copy()

# Label Encoding for non-numeric columns
label_encoders = {}

for col in non_numeric_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    label_encoders[col] = le
    print(f"Encoded {col}: {len(le.classes_)} unique categories")

print("\nAll columns are now numeric!")
print(f"Final shape: {df_encoded.shape}")

In [ ]:
# Verify no missing values and all numeric
print("Final Data Check:")
print("="*50)
print(f"Missing values: {df_encoded.isnull().sum().sum()}")
print(f"Non-numeric columns: {len(df_encoded.select_dtypes(exclude=[np.number]).columns)}")
print(f"Total features: {df_encoded.shape[1]}")
print(f"Total samples: {df_encoded.shape[0]}")

df_encoded.head()

## Part 3: Feature Standardization

PCA is affected by the scale of features, so we need to standardize the data:
- Mean = 0
- Standard Deviation = 1

Formula: $z = \frac{x - \mu}{\sigma}$

In [ ]:
# TO DO: Standardize the features
# Use StandardScaler to normalize the data

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the data
X_scaled = scaler.fit_transform(df_encoded)

print("Data standardized successfully!")
print(f"Shape: {X_scaled.shape}")
print(f"\nMean of each feature (should be ~0):")
print(np.round(X_scaled.mean(axis=0), 6)[:5], "...")
print(f"\nStandard deviation of each feature (should be ~1):")
print(np.round(X_scaled.std(axis=0), 6)[:5], "...")

## TASK 1: Implement PCA from Scratch

### Steps:
1. Calculate the covariance matrix
2. Compute eigenvalues and eigenvectors
3. Sort eigenvalues in descending order
4. Select principal components
5. Project data onto principal components

### Step 1: Calculate Covariance Matrix

The covariance matrix shows how features vary together:

$\text{Cov}(X, Y) = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})$

For a matrix: $C = \frac{1}{n-1} X^T X$ (when X is centered)

In [ ]:
# TO DO: Calculate the covariance matrix
# Hint: Use np.cov() with rowvar=False, or calculate manually using the formula

# Method 1: Using NumPy
covariance_matrix = np.cov(X_scaled, rowvar=False)

# Method 2: Manual calculation (uncomment to use)
# n_samples = X_scaled.shape[0]
# covariance_matrix = (X_scaled.T @ X_scaled) / (n_samples - 1)

print("Covariance Matrix:")
print(f"Shape: {covariance_matrix.shape}")
print(f"\nFirst 5x5 elements:")
print(np.round(covariance_matrix[:5, :5], 4))

# Visualize covariance matrix
plt.figure(figsize=(12, 10))
sns.heatmap(covariance_matrix, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Covariance Matrix Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

### Step 2: Compute Eigenvalues and Eigenvectors

Eigendecomposition: $Cv = \lambda v$

Where:
- $C$ is the covariance matrix
- $\lambda$ are eigenvalues (variance along principal component)
- $v$ are eigenvectors (direction of principal component)

In [ ]:
# TO DO: Compute eigenvalues and eigenvectors
# Hint: Use np.linalg.eig() or np.linalg.eigh() for symmetric matrices

# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

print("Eigendecomposition completed!")
print(f"Number of eigenvalues: {len(eigenvalues)}")
print(f"Eigenvector matrix shape: {eigenvectors.shape}")
print(f"\nFirst 5 eigenvalues (before sorting):")
print(eigenvalues[:5])

### Step 3: Sort Eigenvalues and Eigenvectors in Descending Order

Principal components are ordered by the amount of variance they explain.
We need to sort eigenvalues from largest to smallest.

In [ ]:
# TO DO: Sort eigenvalues and eigenvectors in descending order
# Hint: Use np.argsort() with [::-1] to reverse the order

# Get indices that would sort eigenvalues in descending order
sorted_indices = np.argsort(eigenvalues)[::-1]

# Sort eigenvalues and eigenvectors
eigenvalues_sorted = eigenvalues[sorted_indices]
eigenvectors_sorted = eigenvectors[:, sorted_indices]

print("Eigenvalues and eigenvectors sorted!")
print(f"\nTop 10 eigenvalues (sorted):")
for i, ev in enumerate(eigenvalues_sorted[:10], 1):
    print(f"PC{i}: {ev:.6f}")

### Step 4: Calculate Explained Variance Ratio

Explained variance ratio shows how much information (variance) each principal component captures:

$\text{Explained Variance Ratio}_i = \frac{\lambda_i}{\sum_{j=1}^{n} \lambda_j}$

In [ ]:
# TO DO: Calculate explained variance ratio for each principal component

# Calculate explained variance ratio
total_variance = np.sum(eigenvalues_sorted)
explained_variance_ratio = eigenvalues_sorted / total_variance

# Calculate cumulative explained variance
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Create a detailed table
variance_df = pd.DataFrame({
    'PC': [f'PC{i+1}' for i in range(len(eigenvalues_sorted))],
    'Eigenvalue': eigenvalues_sorted,
    'Variance Ratio': explained_variance_ratio * 100,
    'Cumulative Variance': cumulative_variance_ratio * 100
})

print("Explained Variance Analysis:")
print("="*70)
print(variance_df.head(15).to_string(index=False))
print("\n" + "="*70)

In [ ]:
# Visualize explained variance
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scree plot
axes[0].bar(range(1, min(21, len(eigenvalues_sorted)+1)), 
            explained_variance_ratio[:20] * 100, 
            alpha=0.7, color='steelblue')
axes[0].set_xlabel('Principal Component', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Explained Variance (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Scree Plot: Variance Explained by Each PC', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Cumulative variance plot
axes[1].plot(range(1, len(cumulative_variance_ratio)+1), 
             cumulative_variance_ratio * 100, 
             marker='o', linewidth=2, markersize=4, color='darkgreen')
axes[1].axhline(y=95, color='r', linestyle='--', label='95% Variance', linewidth=2)
axes[1].axhline(y=90, color='orange', linestyle='--', label='90% Variance', linewidth=2)
axes[1].set_xlabel('Number of Components', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Cumulative Explained Variance (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Cumulative Variance Explained', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## TASK 2: Dynamic Selection of Principal Components

Select the number of components based on explained variance threshold.

In [ ]:
# TO DO: Implement dynamic component selection
# Select components that explain at least 95% of variance

variance_threshold = 0.95  # 95% variance retention

# Find the number of components needed
n_components = np.argmax(cumulative_variance_ratio >= variance_threshold) + 1

print(f"Dynamic Component Selection:")
print("="*70)
print(f"Variance threshold: {variance_threshold*100}%")
print(f"Number of components selected: {n_components}")
print(f"Total features in original dataset: {X_scaled.shape[1]}")
print(f"Dimensionality reduction: {X_scaled.shape[1]} → {n_components}")
print(f"Reduction percentage: {(1 - n_components/X_scaled.shape[1])*100:.2f}%")
print(f"Actual variance explained: {cumulative_variance_ratio[n_components-1]*100:.2f}%")
print("="*70)

### Step 5: Project Data onto Principal Components

Transform the data to the new feature space:

$X_{\text{transformed}} = X \cdot W$

Where $W$ is the matrix of selected eigenvectors (principal components)

In [ ]:
# TO DO: Project the data onto the selected principal components

# Select the top n_components eigenvectors
principal_components = eigenvectors_sorted[:, :n_components]

print(f"Principal components matrix shape: {principal_components.shape}")

# Project the data
X_pca = X_scaled @ principal_components

print(f"Transformed data shape: {X_pca.shape}")
print(f"\nOriginal shape: {X_scaled.shape}")
print(f"Reduced shape: {X_pca.shape}")
print(f"\nFirst 5 samples in PC space:")
print(X_pca[:5, :min(5, n_components)])

In [ ]:
# Create DataFrame with PC values
pc_columns = [f'PC{i+1}' for i in range(n_components)]
df_pca = pd.DataFrame(X_pca, columns=pc_columns)

print("PCA DataFrame created!")
print(f"\nStatistics of Principal Components:")
df_pca.describe()

## TASK 3: Visualization - Before and After PCA

Compare the original feature space with the transformed principal component space.

In [ ]:
# TO DO: Create visualizations comparing before and after PCA

# Select two features from original data for visualization
feature_1_idx = 0
feature_2_idx = 1

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# BEFORE PCA: Original feature space
axes[0].scatter(X_scaled[:, feature_1_idx], X_scaled[:, feature_2_idx], 
                alpha=0.6, s=50, c='steelblue', edgecolors='black', linewidth=0.5)
axes[0].set_xlabel(f'Feature {feature_1_idx+1}: {df_encoded.columns[feature_1_idx]}', 
                   fontsize=12, fontweight='bold')
axes[0].set_ylabel(f'Feature {feature_2_idx+1}: {df_encoded.columns[feature_2_idx]}', 
                   fontsize=12, fontweight='bold')
axes[0].set_title('BEFORE PCA: Original Feature Space', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].axhline(y=0, color='k', linewidth=0.5)
axes[0].axvline(x=0, color='k', linewidth=0.5)

# AFTER PCA: Principal component space
axes[1].scatter(X_pca[:, 0], X_pca[:, 1], 
                alpha=0.6, s=50, c='darkgreen', edgecolors='black', linewidth=0.5)
axes[1].set_xlabel(f'PC1 ({explained_variance_ratio[0]*100:.2f}% variance)', 
                   fontsize=12, fontweight='bold')
axes[1].set_ylabel(f'PC2 ({explained_variance_ratio[1]*100:.2f}% variance)', 
                   fontsize=12, fontweight='bold')
axes[1].set_title('AFTER PCA: Principal Component Space', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].axhline(y=0, color='k', linewidth=0.5)
axes[1].axvline(x=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

print("\nVisualization Analysis:")
print("="*70)
print("BEFORE PCA:")
print(f"  - Shows relationship between two original features")
print(f"  - Data may be correlated and have redundant information")
print(f"\nAFTER PCA:")
print(f"  - PC1 captures {explained_variance_ratio[0]*100:.2f}% of total variance")
print(f"  - PC2 captures {explained_variance_ratio[1]*100:.2f}% of total variance")
print(f"  - PCs are uncorrelated (orthogonal to each other)")
print(f"  - Data is rotated to align with directions of maximum variance")
print("="*70)

### Interpretation of PCA Transformation:

**TO DO: Write your interpretation here**

1. **Data Structure Preservation:**
   - [Explain how the overall structure of data points is preserved]
   - [Discuss whether clusters or patterns remain visible]

2. **Variance Explanation:**
   - [Explain what PC1 and PC2 represent]
   - [Discuss how much variance is captured by these two components]

3. **Transformation Effects:**
   - [Describe how PCA has rotated/transformed the data]
   - [Explain the benefits of this transformation for your specific dataset]

In [ ]:
# Additional visualization: 3D plot if you have at least 3 PCs
if n_components >= 3:
    from mpl_toolkits.mplot3d import Axes3D
    
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
                        c=X_pca[:, 0], cmap='viridis', 
                        s=50, alpha=0.6, edgecolors='black', linewidth=0.5)
    
    ax.set_xlabel(f'PC1 ({explained_variance_ratio[0]*100:.2f}%)', fontweight='bold')
    ax.set_ylabel(f'PC2 ({explained_variance_ratio[1]*100:.2f}%)', fontweight='bold')
    ax.set_zlabel(f'PC3 ({explained_variance_ratio[2]*100:.2f}%)', fontweight='bold')
    ax.set_title('3D Visualization of First 3 Principal Components', 
                 fontsize=14, fontweight='bold')
    
    plt.colorbar(scatter, ax=ax, label='PC1 Value')
    plt.tight_layout()
    plt.show()
    
    print(f"First 3 PCs explain {cumulative_variance_ratio[2]*100:.2f}% of total variance")

## TASK 4: Validation and Comparison with Scikit-learn

Verify your implementation by comparing with scikit-learn's PCA.

In [ ]:
# Compare with scikit-learn implementation
print("Validation: Comparing with Scikit-learn PCA")
print("="*70)

# Apply scikit-learn PCA
sklearn_pca = SklearnPCA(n_components=n_components)
X_pca_sklearn = sklearn_pca.fit_transform(X_scaled)

# Compare explained variance ratios
comparison_df = pd.DataFrame({
    'Component': [f'PC{i+1}' for i in range(n_components)],
    'Our Implementation': explained_variance_ratio[:n_components] * 100,
    'Scikit-learn': sklearn_pca.explained_variance_ratio_ * 100,
    'Difference': np.abs(explained_variance_ratio[:n_components] - 
                        sklearn_pca.explained_variance_ratio_) * 100
})

print(comparison_df.to_string(index=False))
print("\n" + "="*70)

# Check if implementations match (allowing for sign differences)
max_difference = comparison_df['Difference'].max()
if max_difference < 0.01:
    print("✓ Implementation is CORRECT! Matches scikit-learn.")
else:
    print(f"⚠ Small differences detected (max: {max_difference:.6f}%)")
    print("  This is normal due to numerical precision and sign ambiguity.")

## TASK 5: Performance Benchmarking

Measure the performance of your implementation.

In [ ]:
# TO DO: Benchmark your PCA implementation

print("Performance Benchmarking")
print("="*70)

# Benchmark custom implementation
start_time = time.time()

# Repeat the PCA steps
cov_matrix = np.cov(X_scaled, rowvar=False)
eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)
sorted_idx = np.argsort(eigenvals)[::-1]
eigenvals_sorted = eigenvals[sorted_idx]
eigenvecs_sorted = eigenvecs[:, sorted_idx]
pcs = eigenvecs_sorted[:, :n_components]
X_transformed = X_scaled @ pcs

custom_time = time.time() - start_time

# Benchmark scikit-learn
start_time = time.time()
sklearn_pca = SklearnPCA(n_components=n_components)
X_sklearn = sklearn_pca.fit_transform(X_scaled)
sklearn_time = time.time() - start_time

print(f"Custom Implementation Time: {custom_time:.6f} seconds")
print(f"Scikit-learn Time: {sklearn_time:.6f} seconds")
print(f"\nSpeed Comparison:")
if custom_time < sklearn_time:
    print(f"  Custom is {sklearn_time/custom_time:.2f}x FASTER")
else:
    print(f"  Scikit-learn is {custom_time/sklearn_time:.2f}x faster")
    print(f"  (This is expected as scikit-learn is highly optimized)")

print("\n" + "="*70)

## Part 4: Analysis and Insights

### Feature Importance in Principal Components

In [ ]:
# Analyze feature contributions to principal components
n_display_pcs = min(5, n_components)
n_top_features = 10

# Create loadings matrix (correlation between original features and PCs)
loadings = eigenvectors_sorted[:, :n_display_pcs]

# Create a DataFrame for better visualization
loadings_df = pd.DataFrame(
    loadings,
    columns=[f'PC{i+1}' for i in range(n_display_pcs)],
    index=df_encoded.columns
)

print("Feature Loadings (Contributions to Principal Components):")
print("="*70)

# Show top contributing features for each PC
for i in range(n_display_pcs):
    pc_name = f'PC{i+1}'
    print(f"\n{pc_name} (explains {explained_variance_ratio[i]*100:.2f}% variance):")
    print("-" * 50)
    
    # Get top positive and negative loadings
    top_features = loadings_df[pc_name].abs().nlargest(n_top_features)
    
    for feature, loading in top_features.items():
        actual_loading = loadings_df.loc[feature, pc_name]
        print(f"  {feature:30s}: {actual_loading:7.4f}")

In [ ]:
# Visualize loadings as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(loadings_df.head(20), cmap='RdBu_r', center=0, 
            annot=True, fmt='.2f', cbar_kws={'label': 'Loading'})
plt.title('Feature Loadings on Principal Components\n(First 20 features)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Principal Component', fontweight='bold')
plt.ylabel('Original Feature', fontweight='bold')
plt.tight_layout()
plt.show()